## 네이버 영화평 감성 분석

In [3]:
!pip install Konlpy> /dev/null

In [4]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [5]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", sep='\t')

In [6]:
print(train_df.shape, test_df.shape)
train_df.head(3)

(150000, 3) (50000, 3)


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


### 1. 데이터 전처리

- 트레인 데이터셋

In [7]:
# Null 데이터가 있는지 확인
train_df.isna().sum()

id          0
document    5
label       0
dtype: int64

In [8]:
# Null 데이터 제거
train_df.dropna(how='any', inplace = True)
train_df.shape

(149995, 3)

In [9]:
# 중복여부 확인
train_df.document.nunique()

146182

In [12]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape

(146182, 3)

In [13]:
# 긍정(1)/ 부정(0) 분포
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터셋

In [14]:
test_df.isna().sum()

id          0
document    3
label       0
dtype: int64

In [15]:
test_df.dropna(how='any', inplace = True)
test_df.shape

(49997, 3)

In [16]:
test_df.document.nunique()

49157

In [17]:
test_df.drop_duplicates(subset = ['document'], inplace = True)
test_df.shape

(49157, 3)

In [18]:
test_df.label.value_counts()

1    24711
0    24446
Name: label, dtype: int64

### 2. 텍스트 전처리
- 트레인 데이터셋

In [21]:
# 한글 이외의 문자는 공백으로 처리 후 strip
train_df.document = train_df.document.str.replace('[^ㄱ-하-ㅣ가-힣]',' ').str.strip()
train_df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠 포스터보고 초딩영화줄 오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [23]:
# '' 만 남은 데이터 --> np.nan 으로 대체한 후 제거
train_df.document.replace('',np.nan,inplace=True)
train_df.document.isna().sum()

747

In [24]:
train_df.dropna(how='any', inplace=True)
train_df.shape

(145435, 3)

- 테스트 데이터셋

In [25]:
test_df.document = test_df.document.str.replace('[^ㄱ-하-ㅣ가-힣]',' ').str.strip()
test_df.document.replace('',np.nan,inplace=True)
test_df.dropna(how='any', inplace=True)
test_df.shape

(48860, 3)

- 전처리가 끝난 데이터는 저장

In [26]:
train_df.to_csv('naver_movie_train_전처리완료.tsv', sep ='\t', index=False)
test_df.to_csv('naver_movie_test_전처리완료.tsv', sep ='\t', index=False)

## 3. 한글처리

In [27]:
from konlpy.tag import Okt
okt = Okt()

In [28]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [29]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [30]:
stop_words = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을','ㅋㅋ','ㅠㅠ','ㅎㅎ']
' '.join([word for word in okt.morphs(text, stem=True) if word not in stop_words])

'교도소 이야기 구먼 솔직하다 재미 없다 평점 조정'

In [31]:
from tqdm.notebook import tqdm

X_train = []
for sentence in tqdm(train_df.document):
  morphs = okt.morphs(sentence, stem = True)
  tmp_str = ' '.join([word for word in morphs if word not in stop_words])
  X_train.append(tmp_str)

  0%|          | 0/145435 [00:00<?, ?it/s]

In [32]:
X_test = []
for sentence in test_df.document:
  morphs = okt.morphs(sentence, stem = True)
  tmp_str = ' '.join([word for word in morphs if word not in stop_words])
  X_test.append(tmp_str)

In [34]:
y_train = train_df.label.values
y_test = test_df.label.values

## 4. 피쳐변환 + 모델 학습/평가

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [35]:
cvect = CountVectorizer()
lr = LogisticRegression(random_state=2922)
pipeline = Pipeline([('CVECT', cvect),('LR',lr)])
pipeline.fit(X_train,y_train)

Pipeline(steps=[('CVECT', CountVectorizer()),
                ('LR', LogisticRegression(random_state=2922))])

In [36]:
pipeline.score(X_test, y_test)

0.8277322963569382

## 5. 실제 데이터 테스트

In [37]:
review1 = '모든 국민이 봤으면 하는 영화입니다.'
review2 = '생각보다 지루하고 별로였네요... 보면서 좀 졸았습니다.'


In [38]:
# 전처리
import re
review1 = re.sub('[^가-힣]', ' ',review1)
review2 = re.sub('[^가-힣]', ' ',review2)

In [39]:
morphs = okt.morphs(review1, stem=True)
review1 =  ' '.join([word for word in morphs if word not in stop_words])
morphs = okt.morphs(review2, stem=True)
review2 =  ' '.join([word for word in morphs if word not in stop_words])

In [40]:
pipeline.predict([review1, review2])

array([1, 0])

### 6. 최적 파라미터 찾기

In [41]:
from sklearn.model_selection import GridSearchCV
params = {
    'CVECT__ngram_range':[(1,1),(1,2)],
    'CVECT__max_df': [0.95, 0.98],
    'LR__C': [1,5]
}

In [42]:
grid_pipe = GridSearchCV(pipeline, params, scoring = 'accuracy', cv=3, n_jobs=-1)
%time grid_pipe.fit(X_train,y_train)

CPU times: user 54.3 s, sys: 17 s, total: 1min 11s
Wall time: 3min 50s


GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('CVECT', CountVectorizer()),
                                       ('LR',
                                        LogisticRegression(random_state=2922))]),
             n_jobs=-1,
             param_grid={'CVECT__max_df': [0.95, 0.98],
                         'CVECT__ngram_range': [(1, 1), (1, 2)],
                         'LR__C': [1, 5]},
             scoring='accuracy')

In [43]:
grid_pipe.best_params_

{'CVECT__max_df': 0.95, 'CVECT__ngram_range': (1, 2), 'LR__C': 1}

In [44]:
grid_pipe.best_estimator_.score(X_test,y_test)

0.8478100695865739